# [如何用Python从海量文本抽取主题？](https://zhuanlan.zhihu.com/p/28992175)

你在工作、学习中是否曾因信息过载叫苦不迭？有一种方法能够替你读海量文章，并将不同的主题和对应的关键词抽取出来，让你谈笑间观其大略。本文使用Python对超过1000条文本做主题抽取，一步步带你体会非监督机器学习LDA方法的魅力。想不想试试呢？

每个现代人，几乎都体会过信息过载的痛苦。文章读不过来，音乐听不过来，视频看不过来。可是现实的压力，使你又不能轻易放弃掉。

## 准备

`pip install jieba
pip install pyldavis`

`pip install pandas,sklearn`

为了处理表格数据，我们依然使用数据框工具Pandas。先调用它。

然后读入我们的数据文件datascience.csv，注意它的编码是中文GB18030，不是Pandas默认设置的编码，所以此处需要显式指定编码类型，以免出现乱码错误。

In [1]:
import pandas as pd
df = pd.read_csv("datascience.csv", encoding='gb18030')
# 之后看看数据框的头几行，以确认读取是否正确。
df.head()

,title,author,content
0,大数据产业迎政策暖风 最新大数据概念股一览,财经热点扒客,大数据产业发展受到国家重视，而大数据已经上升为国家战略，未来发展前景很广阔。大数据产业“十三...
1,Google发布机器学习平台Tensorflow游乐场～带你一起玩神经网络！,硅谷周边,点击上方“硅谷周边”关注我，收到最新的文章哦！昨天，Google发布了Tensorflow游...
2,李克强：中国大数据和云计算产业是开放的,苏州高新区金融办,国务院总理李克强当地时间20日上午在纽约下榻饭店同美国经济、金融、智库、媒体等各界人士座谈，...
3,全峰集团持续挖掘大数据,快递物流网,2016年，全峰集团持续挖掘大数据、云计算、“互联网+”等前沿技术和物流快递的融合，并通过优...
4,第366期【微理工】贵州理工学院召开大数据分析与应用专题分享会,贵州理工学院,贵州理工学院召开大数据分析与应用专题分享会 借“创响中国”贵安站巡回接力活动暨2016贵安大...


In [2]:
#我们看看数据框的长度，以确认数据是否读取完整。
df.shape

(1024, 3)

`(1024, 3)`
行列数都与我们爬取到的数量一致，通过。
# 分词
下面我们需要做一件重要工作——分词

我们首先调用jieba分词包。

我们此次需要处理的，不是单一文本数据，而是1000多条文本数据，因此我们需要把这项工作并行化。这就需要首先编写一个函数，处理单一文本的分词。

有了这个函数之后，我们就可以不断调用它来批量处理数据框里面的全部文本（正文）信息了。你当然可以自己写个循环来做这项工作。但这里我们使用更为高效的apply函数。如果你对这个函数有兴趣，可以点击这段教学视频查看具体的介绍。

下面这一段代码执行起来，可能需要一小段时间。请耐心等候。

In [3]:
import jieba
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))
df["content_cutted"] = df.content.apply(chinese_word_cut)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.232 seconds.
Prefix dict has been built succesfully.


In [4]:
#执行完毕之后，我们需要查看一下，文本是否已经被正确分词。
df.content_cutted.head()

0    大 数据 产业 发展 受到 国家 重视 ， 而 大 数据 已经 上升 为 国家 战略 ， 未...
1    点击 上方 “ 硅谷 周边 ” 关注 我 ， 收到 最新 的 文章 哦 ！ 昨天 ， Goo...
2    国务院 总理 李克强 当地 时间 20 日 上午 在 纽约 下榻 饭店 同 美国 经济 、 ...
3    2016 年 ， 全峰 集团 持续 挖掘 大 数据 、 云 计算 、 “ 互联网 + ” 等...
4    贵州 理工学院 召开 大 数据分析 与 应用 专题 分享 会   借 “ 创响 中国 ” 贵...
Name: content_cutted, dtype: object

In [5]:
#文本向量化
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
n_features = 1000
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df.content_cutted)

我们需要人为设定主题的数量。这个要求让很多人大跌眼镜——我怎么知道这一堆文章里面多少主题？！

别着急。应用LDA方法，指定（或者叫瞎猜）主题个数是必须的。如果你只需要把文章粗略划分成几个大类，就可以把数字设定小一些；相反，如果你希望能够识别出非常细分的主题，就增大主题个数。

对划分的结果，如果你觉得不够满意，可以通过继续迭代，调整主题数量来优化。

这里我们先设定为5个分类试试。

In [6]:
#应用LDA方法
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 5
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

#这一部分工作量较大，程序会执行一段时间，Jupyter Notebook在执行中可能暂时没有响应。等待一会儿就好，不要着急。
lda.fit(tf)

#主题没有一个确定的名称，而是用一系列关键词刻画的。我们定义以下的函数，把每个主题里面的前若干个关键词显示出来：
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

/usr/local/lib/python2.7/dist-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [7]:
#定义好函数之后，我们暂定每个主题输出前20个关键词。
n_top_words = 20

#以下命令会帮助我们依次输出每个主题的关键词表：
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
学习 模型 使用 算法 方法 机器 可视化 神经网络 特征 处理 不同 计算 用户 数据库 系统 如果 分类 训练 一种 基于
Topic #1:
这个 就是 可能 没有 如果 他们 自己 很多 什么 不是 但是 或者 因为 时候 这样 现在 电子 一些 所以 孩子
Topic #2:
企业 平台 服务 管理 互联网 数据分析 公司 产品 用户 业务 行业 客户 金融 创新 实现 价值 系统 能力 工作 需求
Topic #3:
中国 2016 市场 增长 10 城市 用户 2015 关注 行业 其中 30 人口 检索 阅读 大众 投资 全国 美国 20
Topic #4:
人工智能 学习 领域 智能 机器人 机器 人类 公司 深度 研究 未来 识别 已经 系统 计算机 目前 医疗 语音 方面 服务
()


到这里，LDA已经成功帮我们完成了主题抽取。但是我知道你不是很满意，因为结果不够直观。

那咱们就让它直观一些好了。

执行以下命令，会有有趣的事情发生。

In [9]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

/usr/local/lib/python2.7/dist-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      25.852071        1       1  0.149635  0.021451
2      25.798362        1       2 -0.100942  0.102186
1      21.632325        1       3  0.127037 -0.112524
4      14.625269        1       4 -0.005510  0.118440
3      12.091972        1       5 -0.170219 -0.129553, topic_info=     Category         Freq  Term        Total  loglift  logprob
term                                                           
122   Default  2771.000000  人工智能  2771.000000  30.0000  30.0000
364   Default  4194.000000    学习  4194.000000  29.0000  29.0000
148   Default  3220.000000    企业  3220.000000  28.0000  28.0000
615   Default  2178.000000    机器  2178.000000  27.0000  27.0000
586   Default  1450.000000    智能  1450.000000  26.0000  26.0000
616   Default  1109.000000   机器人  1109.000000  25.0000  25.0000
91    Default  1773.000000    中国  1773.000000  24.0000  24.0000
992   Default  2003.000000    领域  2003.000000  23.0000  23.0000
125   Default  1017.000000    人类  1017.000000  22.0000  22.0000
929   Default  2265.000000    这个  2265.000000  21.0000  21.0000
645   Default  1765.000000    模型  1765.000000  20.0000  20.0000
778   Default  1839.000000    算法  1839.000000  19.0000  19.0000
18    Default  1004.000000  2016  1004.000000  18.0000  18.0000
193   Default  2440.000000    公司  2440.000000  17.0000  17.0000
779   Default  1362.000000    管理  1362.000000  16.0000  16.0000
401   Default  1798.000000    就是  1798.000000  15.0000  15.0000
678   Default  1336.000000    深度  1336.000000  14.0000  14.0000
417   Default  1411.000000    市场  1411.000000  13.0000  13.0000
340   Default   814.000000    增长   814.000000  12.0000  12.0000
607   Default  1748.000000    服务  1748.000000  11.0000  11.0000
331   Default   798.000000    城市   798.000000  10.0000  10.0000
424   Default  1734.000000    平台  1734.000000   9.0000   9.0000
129   Default  1067.000000    什么  1067.000000   8.0000   8.0000
1     Default   979.000000    10   979.000000   7.0000   7.0000
85    Default  1121.000000    业务  1121.000000   6.0000   6.0000
360   Default  1663.000000    如果  1663.000000   5.0000   5.0000
454   Default  1239.000000    很多  1239.000000   4.0000   4.0000
845   Default  1589.000000    行业  1589.000000   3.0000   3.0000
368   Default   626.000000    孩子   626.000000   2.0000   2.0000
718   Default   727.000000    电子   727.000000   1.0000   1.0000
...       ...          ...   ...          ...      ...      ...
187    Topic5   302.316730    全国   390.684292   1.8562  -5.1199
609    Topic5   136.545315    期间   177.943139   1.8478  -5.9147
8      Topic5   242.205138    15   317.223026   1.8428  -5.3416
716    Topic5   198.114045    申请   259.964435   1.8409  -5.5425
21     Topic5   163.196166    25   215.494631   1.8346  -5.7364
326    Topic5   236.632356    地区   313.108847   1.8326  -5.3648
340    Topic5   610.327748    增长   814.615688   1.8239  -4.4174
18     Topic5   750.205913  2016  1004.067948   1.8212  -4.2110
22     Topic5   326.256365    30   448.508389   1.7944  -5.0437
17     Topic5   485.246729  2015   708.467452   1.7342  -4.6467
331    Topic5   534.501579    城市   798.602410   1.7111  -4.5500
91     Topic5  1047.305694    中国  1773.569103   1.5859  -3.8774
1      Topic5   591.436906    10   979.981143   1.6077  -4.4488
4      Topic5   255.351191    11   352.936029   1.7890  -5.2887
417    Topic5   640.828649    市场  1411.161873   1.3232  -4.3686
250    Topic5   248.074821    北京   366.565127   1.7222  -5.3176
969    Topic5   311.601217    阅读   562.099252   1.5227  -5.0896
228    Topic5   238.056304    分别   349.847809   1.7276  -5.3588
13     Topic5   296.282344    20   546.958099   1.4996  -5.1400
198    Topic5   382.407822    关注   931.671734   1.2221  -4.8849
906    Topic5   272.897849    超过   500.115946   1.5069  -5.2222
713    Topic5   523.213916    用户  2486.159933   0.5541  -4.5714
203    Topic5   330.011311    其中   825.447658   1.1958  -5.

祝探索旅程愉快！